# Implementation of Theorem 18 for the quadratic form $x^2 + zy^2 = \frac{n}{2}$

## Library Importation

In [56]:
import pandas as pd
import sympy as sp
from IPython.display import display

## Class definitions

### Diophantine solver for the associated quadratic form.

In [57]:
class DiophantineSolver:
    def __init__(self, z: int, max_n: int = 1000, max_solutions_per_n: int = 10):
        self.z = z
        self.max_n = max_n
        self.max_solutions_per_n = max_solutions_per_n
        self.x, self.y = sp.symbols('x y', integer=True)
        self.results_df = pd.DataFrame(columns=["x", "y", "n"])

    def _solve_for_n(self, n):
        eq = sp.Eq(self.x**2 + self.z * self.y**2, n)
        try:
            solutions = sp.diophantine(eq)
            count = 0
            seen = set()

            for sol in solutions:
                if len(sol) != 2:
                    continue
                x_expr, y_expr = sol
                
                for t_val in range(-50, 51): 
                    try:
                        x_val = int(x_expr.subs("t", t_val)) if x_expr.free_symbols else int(x_expr)
                        y_val = int(y_expr.subs("t", t_val)) if y_expr.free_symbols else int(y_expr)
                    except Exception:
                        continue

                    if x_val >= 0 and y_val >= 0 and (x_val, y_val) not in seen:
                        self.results_df.loc[len(self.results_df)] = [x_val, y_val, n]
                        seen.add((x_val, y_val))
                        count += 1
                        if count >= self.max_solutions_per_n:
                            return

        except Exception as e:
            print(f"Error solving for n = {n}: {e}")

    def solve(self):
        self.results_df = pd.DataFrame(columns=["x", "y", "n"])  # reset
        for n in range(1, self.max_n + 1):
            self._solve_for_n(n)
        return self.results_df

### Pell equation solver for $a^2 - 2b^2  = z$ and $u^2 - 2v^2 = -2$

In [58]:
class PellSolver:
    def __init__(self, z: int):
        self.z = z
        self.results = {}

    def solve(self):
        A, B = sp.symbols('A B', integer=True)
        U, V = sp.symbols('U V', integer=True)

        eq1 = sp.Eq(A**2 - 2 * B**2, self.z)
        eq2 = sp.Eq(U**2 - 2 * V**2, -2)

        solns1 = sp.diophantine(eq1)
        solns2 = sp.diophantine(eq2)

        self.results['eq1'] = self._find_minimal_positive_solution(solns1, "eq1")
        self.results['eq2'] = self._find_minimal_positive_solution(solns2, "eq2")

        print(f"Minimal fundamental solution (a,b) for a² - 2b² = {self.z}: {self.results['eq1']}")
        print(f"Minimal fundamental solution (u,v) for u² - 2v² = -2: {self.results['eq2']}")

        return self.results

    def _find_minimal_positive_solution(self, solns, label=""):
        for sol in solns:
            free_syms = set().union(*[expr.free_symbols for expr in sol]) if hasattr(sol[0], 'free_symbols') else set()
        
            if not free_syms:
                candidate = tuple(int(expr) for expr in sol)
                if all(x > 0 for x in candidate):
                    return candidate
            
            for t_val in [1, -1]:
                subs = {sym: t_val for sym in free_syms} if free_syms else {}
                try:
                    candidate = tuple(int(expr.subs(subs)) for expr in sol)
                    if all(x > 0 for x in candidate):
                        return candidate
                except Exception:
                    continue
        
        print(f"[Warning] No positive solution found for {label}. Candidates:")
        for sol in solns:
            print(sol)
        return None


## Main function declaration

In [59]:
def run_full_solver(z, max_n, max_solutions_per_n, iterations):
    dio_solver = DiophantineSolver(z=z, max_n=max_n, max_solutions_per_n=max_solutions_per_n)
    dio_solver.solve()

    pell_solver = PellSolver(z=z)
    pell_results = pell_solver.solve()

    if not pell_results.get('eq1') or not pell_results.get('eq2'):
        print("No Pell solutions found.")
        return pd.DataFrame()

    a, b = pell_results['eq1']  
    u, v = pell_results['eq2']  

    T = sp.Matrix([[3,4], [2, 3]])
    uv_vec = sp.Matrix([[u], [v]])

    for i in range(iterations):
        u_n, v_n = map(int, uv_vec)

        triples = []

        for _, row in dio_solver.results_df.iterrows():
            x_k, y_k = int(row['x']), int(row['y'])

            term = y_k * (b * u_n + a * v_n)

            X = term + x_k
            Y = term - x_k
            Z = y_k * (2 * b * v_n + a * u_n)

            triples.append((X, Y, Z))

        col_name = f"(u,v)=({u_n},{v_n})"
        dio_solver.results_df[col_name] = triples

        uv_vec = T @ uv_vec

    return dio_solver.results_df

## Input declaration and dataframe display

In [60]:
def interactive_run():
    try:
        z = int(input("Enter the value for z (integer): "))
        max_n = int(input("Enter max_n (max n to solve for, integer): "))
        max_solutions_per_n = int(input("Enter max solutions per n (integer): "))
        iterations = int(input("Enter number of iterations (integer): "))
    except ValueError:
        print("Please enter valid integer values.")
        return

    df = run_full_solver(z, max_n, max_solutions_per_n, iterations)

    if df.empty:
        print("No solutions found.")
        return

    df = df.drop_duplicates(subset=["x", "y", "n"])
    df = df.sort_values(by=["n", "x", "y"]).reset_index(drop=True)

    # Hide the index by setting the index to empty strings
    df.index = [''] * len(df)

    display(df)

    print(f"\nTotal unique solutions found: {len(df)}")

## Run command

In [61]:
interactive_run()

Minimal fundamental solution (a,b) for a² - 2b² = 17: (7, 4)
Minimal fundamental solution (u,v) for u² - 2v² = -2: (4, 3)


,x,y,n,"(u,v)=(4,3)","(u,v)=(24,17)","(u,v)=(140,99)"
,1,1,18,"(38, 36, 52)","(216, 214, 304)","(1254, 1252, 1772)"
,2,1,21,"(39, 35, 52)","(217, 213, 304)","(1255, 1251, 1772)"
,3,1,26,"(40, 34, 52)","(218, 212, 304)","(1256, 1250, 1772)"
,4,1,33,"(41, 33, 52)","(219, 211, 304)","(1257, 1249, 1772)"
,5,1,42,"(42, 32, 52)","(220, 210, 304)","(1258, 1248, 1772)"
,6,1,53,"(43, 31, 52)","(221, 209, 304)","(1259, 1247, 1772)"
,7,1,66,"(44, 30, 52)","(222, 208, 304)","(1260, 1246, 1772)"
,1,2,69,"(75, 73, 104)","(431, 429, 608)","(2507, 2505, 3544)"
,2,2,72,"(76, 72, 104)","(432, 428, 608)","(2508, 2504, 3544)"
,3,2,77,"(77, 71, 104)","(433, 427, 608)","(2509, 2503, 3544)"



Total unique solutions found: 33
